In [1]:
import pandas as pd
import sys
sys.path.append('/home/tomek/ib_tools')
from saver import ArcticSaver

/home/tomek/.virtualenvs/ib/lib/python3.8/site-packages/arctic/_util.py:6: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
/home/tomek/.virtualenvs/ib/lib/python3.8/site-packages/arctic/store/_pandas_ndarray_store.py:6: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import DataFrame, Series, Panel


In [2]:
saver = ArcticSaver()

Library created, but couldn't enable sharding: no such command: 'enablesharding', full error: {'ok': 0.0, 'errmsg': "no such command: 'enablesharding'", 'code': 59, 'codeName': 'CommandNotFound'}. This is OK if you're not 'admin'


In [3]:
saver.keys()

['all_bars_20200715_15_51_ESU0',
 'all_bars_20200715_15_51_GCQ0',
 'all_bars_20200715_15_51_NQU0',
 'all_bars_20200715_15_51_YM   SEP 20',
 'all_bars_20200719_23_16_ESU0',
 'all_bars_20200719_23_16_GCQ0',
 'all_bars_20200719_23_16_NQU0',
 'all_bars_20200719_23_16_YM_SEP_20',
 'all_bars_20200721_21_30_ESU0',
 'all_bars_20200721_21_30_GCQ0',
 'all_bars_20200721_21_30_NQU0',
 'all_bars_20200721_21_30_YM_SEP_20',
 'all_bars_20200723_14_21_ESU0',
 'all_bars_20200723_14_21_GCQ0',
 'all_bars_20200723_14_21_NQU0',
 'all_bars_20200723_14_21_YM_SEP_20',
 'all_bars_20200723_17_44_ESU0',
 'all_bars_20200723_17_44_GCQ0',
 'all_bars_20200723_17_44_NQU0',
 'all_bars_20200723_17_44_YM_SEP_20',
 'all_bars_20200724_20_08_ESU0',
 'all_bars_20200724_20_08_GCQ0',
 'all_bars_20200724_20_08_GCZ0',
 'all_bars_20200724_20_08_NQU0',
 'all_bars_20200724_20_08_YM_SEP_20',
 'all_bars_20200803_14_47_GCZ0',
 'all_bars_20200803_14_47_YM_SEP_20',
 'all_bars_20200803_15_17_ESU0',
 'all_bars_20200803_15_17_GCZ0',
 'all_

In [39]:
d = saver.read('candles_20201118_11_23_NQZ0').data
del d['weighted_price']
del d['open']
del d['close']
del d['high']
del d['low']
del d['volume']

In [49]:
d['max'] = d.signal.shift().rolling(40).max()
d['min'] = - d.signal.shift().rolling(40).min()
d['_l'] =  d.signal.shift().rolling(40).max() + d.signal.shift().rolling(40).min()

In [50]:
d.tail(50)

,backfill,price,ema_fast,ema_slow,atr,signal,filter,lock,filtered_signal,_l,max,min
date,,,,,,,,,,,,
2020-11-17 20:26:30,True,12026.50,12018.938601,11948.127029,33.623067,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-17 20:59:00,True,12012.00,12016.625734,11949.182991,33.382947,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-17 21:15:30,True,11992.00,12008.417156,11949.890850,33.514988,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-17 21:27:30,True,11963.00,11993.278104,11950.107572,33.847734,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-17 21:39:30,True,11984.50,11990.352069,11950.676150,33.422333,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-17 21:54:30,True,11975.75,11985.484713,11951.090671,33.317535,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-17 22:14:30,True,11984.50,11985.156475,11951.642993,32.618808,0,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-18 02:27:00,True,11948.00,11972.770983,11951.582768,33.349443,-1,1.0,-1.0,0.0,1.0,1.0,-0.0
2020-11-18 05:53:00,True,11972.75,11972.763989,11951.932702,33.894563,0,1.0,-2.0,0.0,0.0,1.0,1.0


In [24]:
df = saver.read('candles_20201026_01_04_NQZ0').data

In [25]:
df.tail(20)

,backfill,open,high,low,close,weighted_price,price,volume,ema_fast,ema_slow,atr,signal,filter,filtered_signal
date,,,,,,,,,,,,,,
2020-10-30 18:28:00,False,11008.25,11013.00,10985.25,11006.00,10998.562371,11006.00,13119,11021.034672,11207.643998,44.074716,0,-1.0,0
2020-10-30 18:37:00,False,11005.50,11012.25,10967.00,10969.25,10988.952119,10969.25,12317,11003.773115,11203.703591,44.120806,-1,-1.0,-1
2020-10-30 18:48:30,False,10969.00,10991.50,10959.25,10984.75,10977.213137,10984.75,12526,10997.432076,11200.084514,43.655284,0,-1.0,0
2020-10-30 18:58:00,False,10984.50,10999.50,10974.75,10993.00,10985.324072,10993.00,12525,10995.954718,11196.661621,42.913900,0,-1.0,0
2020-10-30 19:08:00,False,10992.50,11026.50,10968.50,11015.00,10995.122887,11015.00,12186,11002.303145,11193.658943,43.505512,0,-1.0,0
2020-10-30 19:21:00,False,11014.75,11028.50,10992.50,11005.75,11010.632085,11005.75,12068,11003.452097,11190.553002,43.211178,0,-1.0,0
2020-10-30 19:32:00,False,11006.00,11011.75,10965.00,10968.75,10980.502594,10968.75,12433,10991.884731,11186.886829,43.349956,-1,-1.0,-1
2020-10-30 19:42:00,False,10968.50,10989.50,10951.00,10953.00,10965.912619,10953.00,12271,10978.923154,11183.020922,43.159761,-1,-1.0,-1
2020-10-30 19:49:00,False,10953.00,10987.50,10944.50,10981.50,10967.486695,10981.50,12270,10979.782103,11179.689990,43.153496,0,-1.0,0


In [26]:
import numpy as np

In [27]:
del df['weighted_price']
del df['volume']
del df['open']
del df['high']
del df['low']
del df['close']
df['shifted_signal'] = df.signal.shift()
df['lock'] = (df.signal.shift().rolling(5).max() - df.signal.shift().rolling(5).min()) * -1
df['filtered_signal'] = ((df['signal'] - df['lock']) * ((df['signal'] * df['filter']) == 1))
df.tail(50)

,backfill,price,ema_fast,ema_slow,atr,signal,filter,filtered_signal,shifted_signal,lock
date,,,,,,,,,,
2020-10-30 14:37:30,False,11252.75,11240.190448,11306.079922,54.538689,0,-1.0,0.0,0.0,-0.0
2020-10-30 14:43:00,False,11199.25,11226.543632,11304.314131,55.105799,0,-1.0,0.0,0.0,-0.0
2020-10-30 14:47:00,False,11176.75,11209.945755,11302.205624,54.670278,0,-1.0,0.0,0.0,-0.0
2020-10-30 14:52:00,False,11151.75,11190.547170,11299.718744,54.340071,0,-1.0,0.0,0.0,-0.0
2020-10-30 14:58:00,False,11167.75,11182.948113,11297.537434,53.846342,0,-1.0,0.0,0.0,-0.0
2020-10-30 15:05:00,False,11163.75,11176.548742,11295.326063,53.813153,0,-1.0,0.0,0.0,-0.0
2020-10-30 15:12:00,False,11142.50,11165.199161,11292.800003,53.585186,0,-1.0,0.0,0.0,-0.0
2020-10-30 15:16:30,False,11103.75,11144.716108,11289.675197,53.228904,0,-1.0,0.0,0.0,-0.0
2020-10-30 15:24:00,False,11090.50,11126.644072,11286.383033,52.357182,0,-1.0,0.0,0.0,-0.0


In [8]:
np.invert(df['lock']

SyntaxError: unexpected EOF while parsing (<ipython-input-8-7e31b500346a>, line 1)

In [ ]:
df.tail(50)

In [ ]:
from arctic import Arctic
conn = Arctic('localhost')

In [ ]:
conn.list_libraries()

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient()

In [ ]:
client.list_database_names()

In [ ]:
lib = conn['test']

In [ ]:
df = pd.read_csv('/home/tomek/ib_tools/backtests/backtest_2020-06-29_16-44_20180601_20191231_base.csv')

In [ ]:
df['index'] = pd.to_datetime(df['time'], utc=True)

In [ ]:
df

In [ ]:
from collections import OrderedDict

In [ ]:
data = df.to_dict('records')
data

In [ ]:
data[0]

In [ ]:
data_1 = []
for d in data:
    d.update({'index': d['sys_time']})
    data_1.append(d)

In [ ]:
data_1

In [ ]:
lib.write('blotter', [data[0]])

In [ ]:
lib.read('blotter')

In [ ]:
for row in data:
    lib.write('blotter', [row])

In [ ]:
d=lib.read('blotter')

In [ ]:
d.duplicated()

In [ ]:
lib.write('blotter2', data)

In [ ]:
lib.write('blotter1', data)

In [ ]:
lib.read('blotter1')

In [ ]:
lib.read('blotter2')

In [ ]:
lib.write('FEED::SYMBOL', data_1)

In [ ]:
conn.initialize_library('ver_test')

In [ ]:
ver = conn

In [ ]:
conn.list_libraries()

In [ ]:
from arctic.date import DateRange

In [ ]:
r = DateRange('20190101', '20190630')

In [ ]:
lib.read('blotter1', date_range=r)

In [ ]:
import pymongo

In [ ]:
from pymongo import MongoClient

In [ ]:
client = MongoClient()

In [ ]:
dir(client)

In [ ]:
client.database_names()

In [ ]:
blotter = client.blotter

In [ ]:
dir(blotter)

In [ ]:
for c in blotter.list_collections():
    print(c)

In [ ]:
b = blotter['test_blotter']

In [ ]:
for i in b.find():
    print(i)

In [ ]:
db = client['pymongo_test']

In [ ]:
blotter = db.blotter

In [ ]:
blotter.insert_one(data[0])

In [ ]:
from pprint import pprint
for i in blotter.find():
    pprint(i)

In [ ]:
for i in c:
    print(i)

In [ ]:
blotter.find_one()

In [ ]:
from ib_insync import util

In [ ]:
d = datetime(2018, 8, 20)

In [ ]:
d = pd.to_datetime('20180830')

In [ ]:
blotter.count_documents({})

In [ ]:
blotter.delete_many({})

In [ ]:
for d in data:
    blotter.insert_one(d)

In [ ]:
for i in blotter.find():
    pprint(i)

In [ ]:
n=datetime.now()
u=datetime.utcnow()

In [ ]:
n,u

In [ ]:
util.df([i for i in blotter.find()]).set_index('time')